In [1]:
import torch
import os
from torch.utils.data import DataLoader
from mri_dataset_combined_3 import MRI_Dataset as MRI_Dataset_combined
from mri_dataset import MRI_Dataset
from torchvision.models.efficientnet import efficientnet_v2_m, efficientnet_v2_l
from combined_classifier import DoubleCombinedClassifierLogReg, CombinedClassifierL, CombinedClassifierLogReg, FourCombinedClassifierLogReg
from tqdm import tqdm
from evaluation import compute_metrics_binary
from roc_curve import RocCurveDisplay
from sklearn.metrics import ConfusionMatrixDisplay#, RocCurveDisplay
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score,roc_curve,auc
import scipy.stats
from itertools import cycle
from threshold_tuner import ClassificationThresholdTuner

In [2]:
print(plt.style.available)
plt.style.use('seaborn-v0_8') 

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [3]:
device = torch.device('mps' if torch.mps.is_available() else 'cpu')

In [4]:
fourclass_val_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCI-pMCI-AD-run_1','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [12, 72, 43])
fourclass_test_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [12, 72, 43])

fourclass_val_loader = DataLoader(fourclass_val_dataset, batch_size=16, shuffle=True)
fourclass_test_loader  = DataLoader(fourclass_test_dataset, batch_size=16, shuffle=True)

In [5]:
axial_threeclass_val_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCIpMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 13, orientation= 'AXIAL')
#axial_threeclass_test_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCIpMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 13, orientation= 'AXIAL')

sagittal_threeclass_val_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCIpMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 73, orientation= 'SAGITTAL')
#sagittal_threeclass_test_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCIpMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 73, orientation= 'SAGITTAL')

coronal_threeclass_val_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCIpMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 41, orientation= 'CORONAL')
#coronal_threeclass_test_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCIpMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 41, orientation= 'CORONAL')

axial_threeclass_val_loader = DataLoader(axial_threeclass_val_dataset, batch_size=16, shuffle=True)
sagittal_threeclass_val_loader = DataLoader(sagittal_threeclass_val_dataset, batch_size=16, shuffle=True)
coronal_threeclass_val_loader = DataLoader(coronal_threeclass_val_dataset, batch_size=16, shuffle=True)


In [6]:
axial_twoclass_val_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-sMCI-pMCI','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 6, orientation='AXIAL')

sagittal_twoclass_val_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-sMCI-pMCI','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 58, orientation= 'SAGITTAL')

coronal_twoclass_val_dataset = MRI_Dataset('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-sMCI-pMCI','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= 43, orientation= 'CORONAL')


axial_twoclass_val_loader = DataLoader(axial_twoclass_val_dataset, batch_size=16, shuffle=True)

sagittal_twoclass_val_loader = DataLoader(sagittal_twoclass_val_dataset, batch_size=16, shuffle=True)

coronal_twoclass_val_loader = DataLoader(coronal_twoclass_val_dataset, batch_size=16, shuffle=True)

In [7]:
"""
axial = efficientnet_v2_l(num_classes = 4)
sagittal = efficientnet_v2_l(num_classes = 4)
coronal = efficientnet_v2_l(num_classes = 4)


model = CombinedClassifierL(4, axial, sagittal, coronal)
model.load_state_dict(torch.load('/Users/olath/Downloads/model_4class_202503031601_best.pth', weights_only=True, map_location=torch.device('mps')))
model.to(device)
"""

"\naxial = efficientnet_v2_l(num_classes = 4)\nsagittal = efficientnet_v2_l(num_classes = 4)\ncoronal = efficientnet_v2_l(num_classes = 4)\n\n\nmodel = CombinedClassifierL(4, axial, sagittal, coronal)\nmodel.load_state_dict(torch.load('/Users/olath/Downloads/model_4class_202503031601_best.pth', weights_only=True, map_location=torch.device('mps')))\nmodel.to(device)\n"

In [8]:
axial = efficientnet_v2_m(num_classes = 3)
axial.load_state_dict(torch.load('/Users/olath/Downloads/model_Orientation_ AXIAL Magnitude_ 12 Ops_ 4 Dropout_ 0.4_202503070702_best.pth', weights_only=True, map_location=torch.device('mps')))
sagittal = efficientnet_v2_m(num_classes = 3)
sagittal.load_state_dict(torch.load('/Users/olath/Downloads/model_Orientation_ SAGITTAL Magnitude_ 16 Ops_ 4 Dropout_ 0.3_202503071355_best.pth', weights_only=True, map_location=torch.device('mps')))
coronal = efficientnet_v2_m(num_classes = 3)
coronal.load_state_dict(torch.load('/Users/olath/Downloads/model_Orientation_ CORONAL Magnitude_ 14 Ops_ 3 Dropout_ 0.4_202503070848_best.pth', weights_only=True, map_location=torch.device('mps')))


<All keys matched successfully>

In [ ]:
axial = efficientnet_v2_m(num_classes = 2)
axial.load_state_dict(torch.load('/Users/olath/Downloads/model_Orientation_ AXIAL Magnitude_ 16 Ops_ 3 Dropout_ 0.2_202503120213_best.pth', weights_only=True, map_location=torch.device('mps')))
sagittal = efficientnet_v2_m(num_classes = 2)
sagittal.load_state_dict(torch.load('/Users/olath/Downloads/model_Orientation_ SAGITTAL Magnitude_ 16 Ops_ 2 Dropout_ 0.2_202503120244_best.pth', weights_only=True, map_location=torch.device('mps')))
coronal = efficientnet_v2_m(num_classes = 2)
coronal.load_state_dict(torch.load('/Users/olath/Downloads/model_Orientation_ CORONAL Magnitude_ 14 Ops_ 2 Dropout_ 0.2_202503120101_best.pth', weights_only=True, map_location=torch.device('mps')))

<All keys matched successfully>

In [5]:
def create_lists():

    global AUC_list; AUC_list = list()
    global accuracy_list; accuracy_list = list()
    global top2accuracy_list; top2accuracy_list = list()
    global f1_list; f1_list = list()
    global precision_list; precision_list = list()
    global recall_list; recall_list = list()

    global auc_CN_list; auc_CN_list = list()
    global f1_CN_list; f1_CN_list = list()
    global precision_CN_list; precision_CN_list = list()
    global recall_CN_list; recall_CN_list = list()

    global auc_sMCI_list; auc_sMCI_list = list()
    global f1_sMCI_list; f1_sMCI_list = list()
    global precision_sMCI_list; precision_sMCI_list = list()
    global recall_sMCI_list; recall_sMCI_list = list()

    global auc_pMCI_list; auc_pMCI_list = list()
    global f1_pMCI_list; f1_pMCI_list = list()
    global precision_pMCI_list; precision_pMCI_list = list()
    global recall_pMCI_list; recall_pMCI_list = list()

    global auc_AD_list; auc_AD_list = list()
    global f1_AD_list; f1_AD_list = list()
    global precision_AD_list; precision_AD_list = list()
    global recall_AD_list; recall_AD_list = list()

    global y_true_list; y_true_list = list()
    global y_pred_proba_list; y_pred_proba_list = list()

    global threshold_list; threshold_list = list()

In [ ]:
def evaluate(model, fourclass_val_loader, fourclass_test_loader):

    model = model
    model.to(device)
    model.eval()

    predicted_probas = torch.Tensor().to(device)
    true_labels = torch.Tensor().to(device)

    with torch.no_grad():
        for batch in tqdm(fourclass_val_loader, desc="Finding best thresholds"):
        # for batch in data_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)

            true_labels = torch.cat((true_labels,y),0)
            predicted_probas = torch.cat((predicted_probas,y_hat),0)

    tuner = ClassificationThresholdTuner()

    pred_proba = torch.softmax(predicted_probas, 1).cpu().numpy()

    from sklearn.metrics import f1_score

    best_threshold = tuner.tune_threshold(
    y_true=list(map(int, true_labels.cpu().numpy())), 
    target_classes=['0', '1', '2', '3'],
    y_pred_proba=pred_proba,
    metric=f1_score,
    average='macro',
    higher_is_better=True,
    max_iterations=500,
    default_class='0',
    plot_thresholds= True
)
    print(best_threshold)

    predicted_probas = torch.Tensor().to(device)
    true_labels = torch.Tensor().to(device)

    with torch.no_grad():
        for batch in tqdm(fourclass_test_loader, desc="Evaluating"):
        # for batch in data_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)

            true_labels = torch.cat((true_labels,y),0)
            predicted_probas = torch.cat((predicted_probas,y_hat),0)

    pred_proba = torch.softmax(predicted_probas, 1).cpu().numpy()

    tuned_pred = tuner.get_predictions(
    ['0', '1', '2', '3'],
    pred_proba, 
    '0', 
    best_threshold)
    
    validation_metrics = compute_metrics_binary(true_labels, predicted_probas, 4, verbose = 0, pred_labels = tuned_pred)

    return [validation_metrics, best_threshold]


In [7]:
def add_to_lists(validation_metrics_list):

    validation_metrics = validation_metrics_list[0]

    AUC_list.append(validation_metrics['auc'])
    accuracy_list.append(validation_metrics['accuracy'])
    top2accuracy_list.append(validation_metrics['top_2'])
    f1_list.append(validation_metrics['f1score'])
    precision_list.append(validation_metrics['precision'])
    recall_list.append(validation_metrics['recall'])

    auc_CN_list.append(validation_metrics['auc_perclass'][0])
    f1_CN_list.append(validation_metrics['f1_perclass'][0])
    precision_CN_list.append(validation_metrics['precision_perclass'][0])
    recall_CN_list.append(validation_metrics['recall_perclass'][0])

    auc_sMCI_list.append(validation_metrics['auc_perclass'][1])
    f1_sMCI_list.append(validation_metrics['f1_perclass'][1])
    precision_sMCI_list.append(validation_metrics['precision_perclass'][1])
    recall_sMCI_list.append(validation_metrics['recall_perclass'][1])

    auc_pMCI_list.append(validation_metrics['auc_perclass'][2])
    f1_pMCI_list.append(validation_metrics['f1_perclass'][2])
    precision_pMCI_list.append(validation_metrics['precision_perclass'][2])
    recall_pMCI_list.append(validation_metrics['recall_perclass'][2])

    auc_AD_list.append(validation_metrics['auc_perclass'][3])
    f1_AD_list.append(validation_metrics['f1_perclass'][3])
    precision_AD_list.append(validation_metrics['precision_perclass'][3])
    recall_AD_list.append(validation_metrics['recall_perclass'][3])


    y_true_list.extend(validation_metrics['roc_auc'][0])
    y_pred_proba_list.extend(validation_metrics['roc_auc'][1])

    threshold_list.append(validation_metrics_list[1])
    

In [8]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h, np.std(a)

In [9]:
def print_metrics():

    print("Mean AUC:            %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(AUC_list))
    print("Mean Accuracy:       %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(accuracy_list))
    print("Mean Top 2 Accuracy: %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(top2accuracy_list))
    print("Mean F1:             %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(f1_list))
    print("Mean Precision:      %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(precision_list))
    print("Mean Recall:         %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(recall_list))
    print("---------------------------------")
    print("Mean CN AUC:         %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(auc_CN_list))
    print("Mean CN F1:          %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(f1_CN_list))
    print("Mean CN Precision:   %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(precision_CN_list))
    print("Mean CN Recall:      %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(recall_CN_list))
    print("Mean CN Threshold:   %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval([sublist[0] for sublist in threshold_list]))
    print("---------------------------------")
    print("Mean sMCI AUC:       %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(auc_sMCI_list))
    print("Mean sMCI F1:        %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(f1_sMCI_list))
    print("Mean sMCI Precision: %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(precision_sMCI_list))
    print("Mean sMCI Recall:    %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(recall_sMCI_list))
    print("Mean sMCI Threshold: %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval([sublist[1] for sublist in threshold_list]))
    print("---------------------------------")
    print("Mean pMCI AUC:       %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(auc_pMCI_list))
    print("Mean pMCI F1:        %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(f1_pMCI_list))
    print("Mean pMCI Precision: %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(precision_pMCI_list))
    print("Mean pMCI Recall:    %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(recall_pMCI_list))
    print("Mean pMCI Threshold: %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval([sublist[2] for sublist in threshold_list]))
    print("---------------------------------")
    print("Mean AD AUC:         %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(auc_AD_list))
    print("Mean AD F1:          %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(f1_AD_list))
    print("Mean AD Precision:   %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(precision_AD_list))
    print("Mean AD Recall:      %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval(recall_AD_list))
    print("Mean AD Threshold:   %1.3f ±%1.3f STD: %1.3f" %mean_confidence_interval([sublist[3] for sublist in threshold_list]))
    print("---------------------------------")

In [10]:
def eval_32class(path, num_of_models):

    from mri_dataset_combined_2 import MRI_Dataset as MRI_Dataset_combined

    create_lists()

    fourclass_val_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [13, 73, 41, 6, 58, 43])

    fourclass_val_loader  = DataLoader(fourclass_val_dataset, batch_size=16, shuffle=True)

    fourclass_test_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [13, 73, 41, 6, 58, 43])

    fourclass_test_loader  = DataLoader(fourclass_test_dataset, batch_size=16, shuffle=True)

    for i in range(0, num_of_models):

        axial1 = efficientnet_v2_m(num_classes = 3)
        sagittal1 = efficientnet_v2_m(num_classes = 3)
        coronal1 = efficientnet_v2_m(num_classes = 3)
        axial2 = efficientnet_v2_m(num_classes = 2)
        sagittal2 = efficientnet_v2_m(num_classes = 2)
        coronal2 = efficientnet_v2_m(num_classes = 2)

        model = DoubleCombinedClassifierLogReg(4, axial1, sagittal1, coronal1, axial2, sagittal2, coronal2, dropout = 0.0, num_outputs = 64)
        model.load_state_dict(torch.load(os.path.join(path, (str(i)+'.pth')), weights_only=True, map_location=torch.device('mps')))

        add_to_lists(evaluate(model, fourclass_val_loader , fourclass_test_loader))

    print_metrics()



In [11]:
def eval_4class(path, num_of_models):

    from mri_dataset_combined import MRI_Dataset as MRI_Dataset_combined

    create_lists()

    fourclass_val_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [12, 72, 43])

    fourclass_val_loader  = DataLoader(fourclass_val_dataset, batch_size=16, shuffle=True)

    fourclass_test_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [12, 72, 43])

    fourclass_test_loader  = DataLoader(fourclass_test_dataset, batch_size=16, shuffle=True)

    for i in range(0, num_of_models):

        axial = efficientnet_v2_l(num_classes = 4)
        sagittal = efficientnet_v2_l(num_classes = 4)
        coronal = efficientnet_v2_l(num_classes = 4)

        model = CombinedClassifierLogReg(4, axial, sagittal, coronal, dropout = 0.0, num_outputs = 32)
        model.load_state_dict(torch.load(os.path.join(path, (str(i)+'.pth')), weights_only=True, map_location=torch.device('mps')))

        add_to_lists(evaluate(model, fourclass_val_loader, fourclass_test_loader))

    print_metrics()


In [12]:
def eval_1vrest_class(path, num_of_models):

    from mri_dataset_combined_3 import MRI_Dataset as MRI_Dataset_combined

    create_lists()

    fourclass_val_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/val-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [12, 73, 41, 11, 73, 54, 11, 71, 40, 25, 53, 57])

    fourclass_val_loader  = DataLoader(fourclass_val_dataset, batch_size=16, shuffle=True)

    fourclass_test_dataset = MRI_Dataset_combined('/Users/olath/Documents/GitHub/Master-thesis/Datasets/test-CN-sMCI-pMCI-AD','/Users/olath/Documents/ADNI_SLICED_RESCALED/', slice= [12, 73, 41, 11, 73, 54, 11, 71, 40, 25, 53, 57])

    fourclass_test_loader  = DataLoader(fourclass_test_dataset, batch_size=16, shuffle=True)


    for i in range(0, num_of_models):
        
        axial1 = efficientnet_v2_m(num_classes = 2)
        sagittal1 = efficientnet_v2_m(num_classes = 2)
        coronal1 = efficientnet_v2_m(num_classes = 2)
        
        axial2 = efficientnet_v2_m(num_classes = 2)
        sagittal2 = efficientnet_v2_m(num_classes = 2)
        coronal2 = efficientnet_v2_m(num_classes = 2)

        axial3 = efficientnet_v2_m(num_classes = 2)
        sagittal3 = efficientnet_v2_m(num_classes = 2)
        coronal3 = efficientnet_v2_m(num_classes = 2)
        
        axial4 = efficientnet_v2_m(num_classes = 2)
        sagittal4 = efficientnet_v2_m(num_classes = 2)
        coronal4 = efficientnet_v2_m(num_classes = 2)
        
        model = FourCombinedClassifierLogReg(4, [(axial1), (sagittal1), (coronal1), (axial2), (sagittal2), (coronal2), (axial3), (sagittal3), (coronal3), (axial4), (sagittal4), (coronal4)], dropout= 0.4, num_outputs = 256)
        model.load_state_dict(torch.load(os.path.join(path, (str(i)+'.pth')), weights_only=True, map_location=torch.device('mps')))

        add_to_lists(evaluate(model, fourclass_val_loader,fourclass_test_loader))

    print_metrics()

In [13]:
eval_32class('/Users/olath/Downloads/32class_weights', 10)

100%|██████████| 500/500 [27:18<00:00,  3.28s/it]


[0.0, 0.0, 0.26, 0.28891591039999986]


  1%|          | 4/500 [00:15<31:17,  3.79s/it]


[0.0, 0.34555346449999946, 0.0, 0.0]


  3%|▎         | 14/500 [00:48<28:02,  3.46s/it]


KeyboardInterrupt: 

In [77]:
eval_4class('/Users/olath/Downloads/4class_weights', 10)

  4%|▍         | 2/50 [00:02<01:09,  1.46s/it]


[0.0, 0.2, 0.2, 0.29840000000000005]


  6%|▌         | 3/50 [00:03<00:58,  1.25s/it]


[0.0, 0.29136, 0.2, 0.30000000000000004]


  4%|▍         | 2/50 [00:04<01:56,  2.42s/it]


...

[0.0, 0.2, 0.2, 0.30000000000000004]


  4%|▍         | 2/50 [00:01<00:28,  1.68it/s]


[0.0, 0.2, 0.2, 0.30000000000000004]


Evaluating: 100%|██████████| 62/62 [00:14<00:00,  4.38it/s]

Mean AUC:            0.712 ±0.002 STD: 0.002
Mean Accuracy:       0.430 ±0.005 STD: 0.007
Mean Top 2 Accuracy: 0.736 ±0.007 STD: 0.009
Mean F1:             0.416 ±0.005 STD: 0.006
Mean Precision:      0.437 ±0.006 STD: 0.008
Mean Recall:         0.416 ±0.006 STD: 0.008
---------------------------------
Mean CN AUC:         0.736 ±0.004 STD: 0.005
Mean CN F1:          0.524 ±0.010 STD: 0.013
Mean CN Precision:   0.463 ±0.013 STD: 0.017
Mean CN Recall:      0.607 ±0.030 STD: 0.040
Mean CN Threshold:   0.000 ±0.000 STD: 0.000
---------------------------------
Mean sMCI AUC:       0.614 ±0.004 STD: 0.006
Mean sMCI F1:        0.362 ±0.022 STD: 0.029
Mean sMCI Precision: 0.397 ±0.013 STD: 0.017
Mean sMCI Recall:    0.339 ±0.040 STD: 0.052
Mean sMCI Threshold: 0.219 ±0.029 STD: 0.038
---------------------------------
Mean pMCI AUC:       0.696 ±0.004 STD: 0.005
Mean pMCI F1:        0.321 ±0.026 STD: 0.034
Mean pMCI Precision: 0.313 ±0.010 STD: 0.013
Mean pMCI Recall:    0.335 ±0.047 STD: 0.06

In [78]:
eval_1vrest_class('/Users/olath/Downloads/1vrest_weights', 10)

  4%|▍         | 2/50 [00:05<02:10,  2.71s/it]


[0.0, 0.2996000000000001, 0.2, 0.29992]


 12%|█▏        | 6/50 [00:11<01:24,  1.93s/it]


[0.0, 0.299937, 0.2, 0.29234826752000004]


  4%|▍         | 2/50 [00:07<03:09,  3.94s/it]


...

[0.0, 0.2982180000000001, 0.2, 0.30000000000000004]


  8%|▊         | 4/50 [00:08<01:38,  2.13s/it]


[0.0, 0.29933499999999996, 0.2, 0.294158577]


Evaluating: 100%|██████████| 62/62 [00:32<00:00,  1.90it/s]

Mean AUC:            0.735 ±0.002 STD: 0.002
Mean Accuracy:       0.444 ±0.007 STD: 0.009
Mean Top 2 Accuracy: 0.755 ±0.006 STD: 0.007
Mean F1:             0.413 ±0.011 STD: 0.015
Mean Precision:      0.486 ±0.007 STD: 0.009
Mean Recall:         0.408 ±0.009 STD: 0.012
---------------------------------
Mean CN AUC:         0.762 ±0.001 STD: 0.002
Mean CN F1:          0.521 ±0.006 STD: 0.007
Mean CN Precision:   0.485 ±0.007 STD: 0.009
Mean CN Recall:      0.563 ±0.016 STD: 0.021
Mean CN Threshold:   0.000 ±0.000 STD: 0.000
---------------------------------
Mean sMCI AUC:       0.634 ±0.005 STD: 0.007
Mean sMCI F1:        0.450 ±0.008 STD: 0.011
Mean sMCI Precision: 0.372 ±0.006 STD: 0.008
Mean sMCI Recall:    0.570 ±0.017 STD: 0.023
Mean sMCI Threshold: 0.298 ±0.001 STD: 0.002
---------------------------------
Mean pMCI AUC:       0.728 ±0.002 STD: 0.002
Mean pMCI F1:        0.293 ±0.010 STD: 0.013
Mean pMCI Precision: 0.494 ±0.019 STD: 0.025
Mean pMCI Recall:    0.209 ±0.012 STD: 0.01

In [19]:
model = axial
model.to(device)
model.eval()

predicted_probas = torch.Tensor().to(device)
true_labels = torch.Tensor().to(device)

with torch.no_grad():
    for batch in tqdm(axial_twoclass_val_loader, desc="Evaluating"):
    # for batch in data_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = model(x)

        true_labels = torch.cat((true_labels,y),0)
        predicted_probas = torch.cat((predicted_probas,y_hat),0)

validation_metrics1 = compute_metrics_binary(true_labels, predicted_probas, 2, verbose = 1)

model = sagittal
model.to(device)
model.eval()

predicted_probas = torch.Tensor().to(device)
true_labels = torch.Tensor().to(device)

with torch.no_grad():
    for batch in tqdm(sagittal_twoclass_val_loader, desc="Evaluating"):
    # for batch in data_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = model(x)

        true_labels = torch.cat((true_labels,y),0)
        predicted_probas = torch.cat((predicted_probas,y_hat),0)

validation_metrics2 = compute_metrics_binary(true_labels, predicted_probas, 2, verbose = 1)

model = coronal
model.to(device)
model.eval()

predicted_probas = torch.Tensor().to(device)
true_labels = torch.Tensor().to(device)

with torch.no_grad():
    for batch in tqdm(coronal_twoclass_val_loader, desc="Evaluating"):
    # for batch in data_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = model(x)

        true_labels = torch.cat((true_labels,y),0)
        predicted_probas = torch.cat((predicted_probas,y_hat),0)

validation_metrics3 = compute_metrics_binary(true_labels, predicted_probas, 2, verbose = 1)

NameError: name 'axial' is not defined

In [20]:
confusiom_matrix = ConfusionMatrixDisplay(validation_metrics['conf_mat'])
confusiom_matrix.plot()


NameError: name 'validation_metrics' is not defined

In [ ]:
def custom_multiclass_curve(y_true, y_score, n_classes, target_names, name, fig, ax):
    #This function is from sklearn website.

    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    y_onehot_test = label_binarizer.fit_transform(y_true)   

    fpr, tpr, roc_auc = dict(), dict(), dict()
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_onehot_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr_grid = np.linspace(0.0, 1.0, 1000)

    # Interpolate all ROC curves at these points
    mean_tpr = np.zeros_like(fpr_grid)

    for i in range(n_classes):
        mean_tpr += np.interp(fpr_grid, fpr[i], tpr[i])  # linear interpolation

    # Average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = fpr_grid
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    #fig, ax = plt.subplots(figsize=(6, 6))
    """
    plt.plot(
        fpr["micro"],
        tpr["micro"],
        label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.3f})",
        #color="deeppink",
        linestyle=":",
        linewidth=4,
    )
    """
    plt.plot(
        fpr["macro"],
        tpr["macro"],
        label=name+f" (AUC = {roc_auc['macro']:.3f})",
        #color="navy",
        #linestyle=":",
        #linewidth=4,
    )
    """
    colors = cycle(["aqua", "darkorange", "cornflowerblue", "navy"])
    for class_id, color in zip(range(n_classes), colors):
        RocCurveDisplay.from_predictions(
            y_onehot_test[:, class_id],
            y_score[:, class_id],
            name=f"ROC curve for {target_names[class_id]}",
            color=color,
            ax=ax,
            plot_chance_level=(class_id == 1),
            #despine=True,
        )
    
    _ = ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title="Extension of Receiver Operating Characteristic\nto One-vs-Rest Multiclass",
    )
    """
    return fig, ax

In [39]:
def multiclass_curve(y_true, y_score, n_classes, target_names, name, values):
    #This function is from sklearn website.

    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    y_onehot_test = label_binarizer.fit_transform(y_true)   

    fpr, tpr, roc_auc = dict(), dict(), dict()
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_onehot_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr_grid = np.linspace(0.0, 1.0, 1000)

    # Interpolate all ROC curves at these points
    mean_tpr = np.zeros_like(fpr_grid)

    for i in range(n_classes):
        mean_tpr += np.interp(fpr_grid, fpr[i], tpr[i])  # linear interpolation

    # Average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = fpr_grid
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    fig, ax = plt.subplots(figsize=(6, 6))
    """
    plt.plot(
        fpr["micro"],
        tpr["micro"],
        label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.3f})",
        #color="deeppink",
        linestyle=":",
        linewidth=4,
    )
    """

    plt.plot(
        fpr["macro"],
        tpr["macro"],
        label=f"macro-average ROC curve (AUC = {values[4]:.3f})",
        #color="navy",
        linestyle=":",
        linewidth=4,
    )

    colors = cycle(["aqua", "darkorange", "cornflowerblue", "navy"])
    for class_id, color in zip(range(n_classes), colors):
        RocCurveDisplay.from_predictions(
            y_onehot_test[:, class_id],
            y_score[:, class_id],
            name=f"ROC curve for {target_names[class_id]}",
            #color=color,
            ax=ax,
            list = values[class_id],
            #plot_chance_level=(class_id == 1),
            #despine=True,
        )

    _ = ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title="One-vs-Rest Multiclass ROC Curves for " + name + ".",
    )

    return fig, ax

In [62]:
#fig, ax = plt.subplots(figsize=(6, 6))

#y_true, y_pred_proba, threshold = validation_metrics['roc_auc']

fig, ax = multiclass_curve(np.array(y_true_list), np.array(y_pred_proba_list), 4, {0:'CN',1:'sMCI', 2:'pMCI', 3:'AD'}, "One-vs-Rest Ensemble", values = [0.762,0.634,0.728,0.816,0.735])
ax.margins(x=0.01, y = 0.01)
default_chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
ax.plot((0, 1), (0, 1), **default_chance_level_line_kw)
plt.legend(loc=0)
#plt.show()
plt.savefig('/Users/olath/Documents/ensemble-1vRest-roclpot(testdata).pdf', dpi = 1200, bbox_inches='tight')
plt.close() 

NameError: name 'multiclass_curve' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

y_true1, y_pred_proba1, threshold = validation_metrics1['roc_auc']

y_true2, y_pred_proba2, threshold = validation_metrics2['roc_auc']

y_true3, y_pred_proba3, threshold = validation_metrics3['roc_auc']

fig, ax = custom_multiclass_curve(y_true1, y_pred_proba1, 3, {0:'CN', 1:'MCI', 2:'AD'}, "Axial", fig, ax)

fig, ax = custom_multiclass_curve(y_true2, y_pred_proba2, 3, {0:'CN', 1:'MCI', 2:'AD'}, "Sagittal", fig, ax)

fig, ax = custom_multiclass_curve(y_true3, y_pred_proba3, 3, {0:'CN', 1:'MCI', 2:'AD'}, "Coronal", fig, ax)

default_chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
ax.plot((0, 1), (0, 1), **default_chance_level_line_kw)
ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title="Macro Average ROC Curves for CN/MCI/AD Submodels",
    )
ax.margins(x=0.01, y = 0.01)
plt.legend(loc=0)
#plt.show()
plt.savefig('/Users/olath/Documents/cn-mci-ad-roclpot(valdata).pdf', dpi = 1200, bbox_inches='tight')
plt.close() 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

label = validation_metrics1['roc_auc'][0]
pred = validation_metrics1['roc_auc'][1]
fpr, tpr, thresh = roc_curve(label, pred[:, 1])
auc = roc_auc_score(label, pred[:, 1])
plt.plot(fpr,tpr,label="Axial"+f" (AUC = {auc:.3f})")

label = validation_metrics2['roc_auc'][0]
pred = validation_metrics2['roc_auc'][1]
fpr, tpr, thresh = roc_curve(label, pred[:, 1])
auc = roc_auc_score(label, pred[:, 1])
plt.plot(fpr,tpr,label="Sagittal"+f" (AUC = {auc:.3f})")

label = validation_metrics3['roc_auc'][0]
pred = validation_metrics3['roc_auc'][1]
fpr, tpr, thresh = roc_curve(label, pred[:, 1])
auc = roc_auc_score(label, pred[:, 1])
plt.plot(fpr,tpr,label="Coronal"+f" (AUC = {auc:.3f})")

default_chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
ax.plot((0, 1), (0, 1), **default_chance_level_line_kw)
ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title="ROC Curves for sMCI/pMCI Submodels",
    )
ax.margins(x=0.01, y = 0.01)

plt.legend(loc=0)
plt.savefig('/Users/olath/Documents/smci-pmci-roclpot(valdata).pdf', dpi = 1200, bbox_inches='tight')
plt.close() 